In [14]:
import time
time_ini=time.time()

In [15]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
comex_df=pd.read_csv('/home/andre301267/git/Pricing-Fertilizer/DB/temp/imp_fert_ncm_quanti.csv')
comex_df.columns

Index(['s_fob_kg', 's_cif_kg', 's_log_frete', 's_log_fob', 's_log_kg', 'year',
       'month', 'ncm', 'Product', 'UF', 'pais', 'urf', 'kg', 'US$_fob',
       'US$_freight', 'US$_insurance', 'log_kg', 'log_fob', 'log_frete',
       'cif_kg', 'fob_kg', 'km', 'mean_log_kg', 'qd_dif_log_kg', 'std_log_kg',
       'z_score_log_kg', 'mean_log_fob', 'qd_dif_log_fob', 'std_log_fob',
       'z_score_log_fob', 'dif_z_score', 'mean_dif_z_score',
       'qd_dif_dif_z_score', 'std_dif_z_score', 'z_score_dif_z_score', 'preço',
       'kg_err'],
      dtype='object')

In [16]:
comex_df=comex_df[['year','month', 'Product', 'UF', 'pais', 'urf', 'kg', 'US$_fob',\
       'US$_freight', 'US$_insurance', 'log_kg', 'log_fob', 'log_frete',\
       'cif_kg', 'fob_kg', 'km', 'preço']]

In [17]:
comex_df.insert(0,'date',pd.to_datetime(comex_df.year.astype(str)+'/'+comex_df.month.astype(str)+'/1'))

In [18]:
comex_df.Product.unique()

array(['DAP arsênio> 6 mg/kg', 'MAP', 'MOP', 'MOP<60', 'NAM', 'NCa', 'NK',
       'NP', 'NPK', 'Nit.Sódio Potássico 15 15', 'Nit.Sódio natural<16,3',
       'Nitratos e fosfatos', 'Outros N', 'Outros Nit.Sódio potássico',
       'Outros Nit.Sódio, naturais', 'Outros SOP',
       'Outros nitratos de sódio', 'Outros potássicos',
       'Outros superfosfatos', 'PK', 'SAM', 'SOP<52', 'Superfosfato>45',
       'Superfosfatos<22', 'Superfosfatos>35', 'UREIA',
       'Ureia, mesmo aquosa'], dtype=object)

In [19]:
Product=['MOP','MAP','UREIA']
UF=['BA']

In [20]:
comex_df=comex_df[(comex_df.Product.isin(Product))&(comex_df.UF.isin(UF))]

In [21]:
# Selecioina o km de maioir média
filtra_km=comex_df[['Product','km']].sort_values(by=['Product','km'],ascending=False)\
    .drop_duplicates(subset='Product')

In [22]:
filtra_km

,Product,km
80461,UREIA,1
17850,MOP,2
5852,MAP,2


In [23]:
comex_df=comex_df.merge(filtra_km)

In [24]:
quanti_cols=['kg','US$_fob','US$_frete']

In [25]:
q=pd.DataFrame()

# 2a Fase: Sparse Matrix - train

In [31]:
comex_df['date']=pd.to_datetime(comex_df.year.astype(str)+'/'+comex_df.month.astype(str)+'/1')

In [32]:
data_ini=comex_df.date.min()
data_fim=comex_df.date.max()

comex_df=comex_df[['year','month','Product','kg']]

comex_df[['year','month','Product']]=comex_df[['year','month','Product']].astype('category')

In [34]:
comex_df=comex_df.groupby(by=['year','month','Product'],observed=False).kg.sum().reset_index()

In [36]:
comex_df.insert(0,'date',pd.to_datetime(comex_df.year.astype(str)+'/'+comex_df.month.astype(str)+'/1'))

comex_df=comex_df[(comex_df.date>=data_ini)&(comex_df.date<=data_fim)]

comex_df[['month','Product']]=comex_df[['month','Product']].astype('category')

comex_df_dum=pd.get_dummies(comex_df,prefix_sep='~')

comex_df_dum=comex_df_dum.assign(oc=1)
comex_df_dum.loc[comex_df_dum.kg==0,'oc']=0

In [37]:
comex_df_dum.to_csv('/home/andre301267/git/Pricing-Fertilizer/DB/temp/imp_fert_select.csv',index=False)